# Supporting Function and Code

In [1]:
import numpy as np
from sklearn import datasets, decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import Isomap, TSNE
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import ticker

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Reshape, UpSampling2D, Conv2DTranspose, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

## Data Generation and Preprocessing

In [3]:
def generate_s_curve(n=3000, noise=0, seed=42):
    """
    Generate a standardized 3D S-Curve dataset.
    
    Parameters:
    n (int): Number of samples.
    noise (float): Standard deviation of Gaussian noise.
    seed (int): Random seed for reproducibility.
    
    Returns:
    tuple: Scaled 3D coordinates and corresponding colour labels.
    """
    x, y = datasets.make_s_curve(n, noise=noise, random_state=seed)

    # Standardise the data
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

    return x_scaled, y

def generate_swiss_roll(n=3000, noise=0, seed=42):
    """
    Generate a standardized 3D Swiss Roll dataset.
    
    Parameters:
    n (int): Number of samples.
    noise (float): Standard deviation of Gaussian noise.
    seed (int): Random seed for reproducibility.
    
    Returns:
    tuple: Scaled 3D coordinates and corresponding colour labels.
    """
    x, y = datasets.make_swiss_roll(n, noise=noise, random_state=seed)

    # Standardise the data
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

    return x_scaled, y

def generate_simulated_data(n=3000, noise=0, seed=42):
    """
    Generate both S-Curve and Swiss Roll datasets.
    
    Parameters:
    n (int): Number of samples.
    noise (float): Standard deviation of Gaussian noise.
    seed (int): Random seed for reproducibility.
    
    Returns:
    tuple: Scaled 3D coordinates and corresponding labels for both datasets.
    """
    x_s, y_s = generate_s_curve(n=n, noise=noise, seed=seed)
    x_swiss, y_swiss = generate_swiss_roll(n=n, noise=noise, seed=seed)

    return (x_s, x_swiss), (y_s, y_swiss)

def load_preprocessed_mnist():
    """
    Load and preprocess the MNIST dataset.
    
    Returns:
    tuple: Training and test images, flattened training and test images, training and test labels, and unique labels.
    """
    # Load and normalise the fashion mnist dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train / 255.
    x_test = x_test / 255.

    # Get distinct label set from training set
    unique_labels = set(y_train)

    # Reshape the dataset for PCA, Isomap and t-SNE
    flatten_x_train = x_train.reshape(-1, 784) 
    flatten_x_test = x_test.reshape(-1, 784)  

    return x_train, x_test, flatten_x_train, flatten_x_test, y_train, y_test, unique_labels

## Visualisation

In [4]:
# Define colour maps
jet_cmap = plt.cm.jet
viridis_cmap = plt.cm.viridis

def plot_3d_datasets(x, y, title, txt, cmap1=viridis_cmap, cmap2=jet_cmap):
    """
    Plot two 3D scatter plots of datasets.
    
    Parameters:
    x (tuple): Two sets of 3D coordinates.
    y (tuple): Two sets of corresponding colour labels.
    title (str): Title for the figure.
    txt (str): Text for figure caption.
    cmap1 (Colormap): Colormap for the first dataset. Default to viridis_cmap.
    cmap2 (Colormap): Colormap for the second dataset. Default to jet_cmap.
    """
    points1, points2 = x
    points_color1, points_color2 = y
    x1, y1, z1 = points1.T
    x2, y2, z2 = points2.T
    
    fig = plt.figure(figsize=(8, 4)) 
    # First 3d scatter subplot
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.scatter(x1, y1, z1, c=points_color1, cmap=cmap1, alpha=0.3)
    ax1.view_init(azim=-60, elev=12)
    ax1.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax1.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax1.zaxis.set_major_locator(ticker.MultipleLocator(1))

    # Second 3d scatter subplot
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.scatter(x2, y2, z2, c=points_color2, cmap=cmap2, alpha=0.3)
    ax2.view_init(azim=-60, elev=12)
    ax2.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax2.zaxis.set_major_locator(ticker.MultipleLocator(1))

    fig.suptitle(title, fontsize='medium')
    fig.text(.5, .05, txt, ha='center')
    fig.tight_layout()
    plt.show()


def visualise_sample_images(x, y, txt='Figure 5: 10 randomly sampled images from the MNIST training dataset.'):
    """
    Visualize a sample of images with labels.
    
    Parameters:
    x (array): Array of images.
    y (array): Array of corresponding labels.
    txt (str): Text for figure caption.
    """
    fig = plt.figure(figsize=(10, 2))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
    for i in range(10):
        ax = fig.add_subplot(1, 10, i + 1, xticks=[], yticks=[])
        ax.imshow(x[i], cmap=plt.cm.binary)
        # label the image with the target value
        ax.text(12, -1.5, str(y[i]))
    fig.text(.5, .1, txt, ha='center')
    fig.tight_layout()
    plt.show() 

## PCA, Isomap and t-SNE

In [5]:
def perform_multiple_dim_reduction(x_data, y_data, dataset, n_comp, n_neighb, cmap, 
                                   txt='Figure 2: Dimensionality reduction results for the S-Curve (Top) and ' 
                                       'Swiss Roll (Bottom) datasets using PCA, Isomap, and t-SNE.'):
    """
    Perform multiple dimensionality reduction techniques (PCA, Isomap & t-SNE) on datasets and plot the results.
    
    Parameters:
    x_data (list): List of datasets to be reduced.
    y_data (list): List of corresponding colour labels for each dataset.
    dataset (list): List of dataset names.
    n_comp (int): Number of components for the dimensionality reduction (used in Isomap & t-SNE).
    n_neighb (int): Number of neighbors for Isomap.
    cmap (list): List of colormaps for each dataset.
    txt (str): Text for figure caption.
    """
    num_dataset = len(x_data)
    fig = plt.figure(figsize=(14,6))
    counter = 0
    # Loop through each dataset
    for i, x in enumerate(x_data):
        y = y_data[i]
        
        # Perform PCA 
        pca = decomposition.PCA()
        pca.fit(x)
        x_pca = pca.transform(x)
        # Visualise the first two principal components for each dataset
        counter+=1
        plt.subplot(num_dataset, 3, counter)
        plt.scatter(x_pca[:, 0], x_pca[:, 1], c=y, cmap=cmap[i], alpha=0.3)
        plt.xlabel('PC1')
        plt.ylabel('PC2')
        plt.title(f'Principal Component Analysis: {dataset[i]}')
        
        # Perform Isomap
        isomap = Isomap(n_components=n_comp, n_neighbors=n_neighb) 
        x_isomap = isomap.fit_transform(x)
        # Visualise the first two dimensions of Isomap embeddings
        counter+=1
        plt.subplot(num_dataset, 3, counter)
        plt.scatter(x_isomap[:, 0], x_isomap[:, 1], c=y, cmap=cmap[i], alpha=0.3)
        plt.xlabel('Dim1')
        plt.ylabel('Dim2')
        plt.title(f'Isomap Embeddings: {dataset[i]}')

        # Perform t-SNE
        tsne = TSNE(n_components=n_comp) 
        x_tsne = tsne.fit_transform(x)
        # Visualise the first two dimensions of t-SNE embeddings
        counter+=1
        plt.subplot(num_dataset, 3, counter)
        plt.scatter(x_tsne[:, 0], x_tsne[:, 1], c=y, cmap=cmap[i], alpha=0.3)
        plt.xlabel('Dim1')
        plt.ylabel('Dim2')
        plt.title(f't-SNE Embeddings: {dataset[i]}')

    fig.text(.5, -0.01, txt, ha='center', fontsize='large')
    fig.tight_layout()
    plt.show()

def perform_mnist_dim_reduction(x, y, n, n_comp, n_neighb, distinct_labels, 
                                txt='Figure 6: Dimensionality reduction results for the MNIST dataset using PCA,'
                                ' Isomap, and t-SNE. Each point represents a MNIST digit in the reduced 2D space,'
                                ' colored by its digit class.'):
    """
    Perform dimensionality reduction techniques (PCA, Isomap & t-SNE) on the MNIST dataset and plot the results.
    
    Parameters:
    x (array): Array of MNIST images (flattened).
    y (array): Array of corresponding labels.
    n (int): Number of samples to visualize.
    n_comp (int): Number of components for the dimensionality reduction (used in Isomap & t-SNE).
    n_neighb (int): Number of neighbors for Isomap.
    cmap (list): List of colormaps for each dataset.
    distinct_labels (set): Set of distinct labels in the dataset.
    txt (str): Text for figure caption.
    """
    fig = plt.figure(figsize=(15,4))

    # Standardise data before performing PCA
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

    # Perform PCA 
    pca = decomposition.PCA()
    pca.fit(x)
    x_pca = pca.transform(x_scaled)
    
    # Visualise the first two principal components for each dataset
    plt.subplot(1, 3, 1)
    for i in distinct_labels:
        plt.scatter(x_pca[:n][y[:n] == i, 0], x_pca[:n][y[:n] == i, 1], label=i, alpha=0.6)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('Principal Component Analysis: MNIST')
    plt.legend()

    # Perform Isomap
    isomap = Isomap(n_components=n_comp, n_neighbors=n_neighb) 
    x_isomap = isomap.fit_transform(x)
    
    # Visualise the first two dimensions of Isomap embeddings
    plt.subplot(1, 3, 2)
    for i in distinct_labels:
        plt.scatter(x_isomap[:n][y[:n] == i, 0], x_isomap[:n][y[:n] == i, 1], label=i, alpha=0.6)
    plt.xlabel('Dim1')
    plt.ylabel('Dim2')
    plt.title('Isomap Embeddings: MNIST')
    plt.legend()
    
    # Perform t-SNE
    tsne = TSNE(n_components=n_comp) 
    x_tsne = tsne.fit_transform(x)

    # Visualise the first two dimensions of t-SNE embeddings
    plt.subplot(1, 3, 3)
    for i in distinct_labels:
        plt.scatter(x_tsne[:n][y[:n] == i, 0], x_tsne[:n][y[:n] == i, 1], label=i, alpha=0.6)
    plt.xlabel('Dim1')
    plt.ylabel('Dim2')
    plt.title('t-SNE Embeddings: MNIST')
    plt.legend()

    fig.text(.5, -0.01, txt, ha='center', fontsize='large')
    fig.tight_layout()
    plt.show()

## Autoencoder

In [6]:
def get_encoder(input_dim, encoded_dim, dataset, seed=5):
    """
    Define the encoder part of an autoencoder.
    
    Parameters:
    input_dim (int): Dimension of the input data.
    encoded_dim (int): Dimension of the encoded representation.
    dataset (str): Name of the dataset.
    seed (int): Random seed for weight initialization.
    
    Returns:
    Model: Encoder model.
    """
    inputs = Input(shape=(input_dim,))
    x = Dense(128, activation='relu', kernel_initializer=GlorotUniform(seed=seed))(inputs)
    x = Dense(64, activation='relu', kernel_initializer=GlorotUniform(seed=seed))(x)
    encoded = Dense(encoded_dim, activation='relu', kernel_initializer=GlorotUniform(seed=seed))(x)
    encoder = Model(inputs, encoded, name=f'{dataset}_encoder')
    return encoder

def get_decoder(input_dim, encoded_dim, dataset, seed=5):
    """
    Define the decoder part of an autoencoder.
    
    Parameters:
    input_dim (int): Dimension of the output data.
    encoded_dim (int): Dimension of the encoded representation.
    dataset (str): Name of the dataset.
    seed (int): Random seed for weight initialization.
    
    Returns:
    Model: Decoder model.
    """
    encoded_inputs = Input(shape=(encoded_dim,))
    x = Dense(64, activation='relu', kernel_initializer=GlorotUniform(seed=seed))(encoded_inputs)
    x = Dense(128, activation='relu', kernel_initializer=GlorotUniform(seed=seed))(x)
    decoded = Dense(input_dim, kernel_initializer=GlorotUniform(seed=seed))(x)
    decoder = Model(encoded_inputs, decoded, name=f'{dataset}_decoder')
    return decoder
    
def get_autoencoder(input_dim, encoded_dim, dataset, seed=5):
    """
    Construct the complete autoencoder by combining encoder and decoder.
    
    Parameters:
    input_dim (int): Dimension of the input data.
    encoded_dim (int): Dimension of the encoded representation.
    dataset (str): Name of the dataset.
    seed (int): Random seed for weight initialization.
    
    Returns:
    Model: Autoencoder model.
    """
    encoder = get_encoder(input_dim, encoded_dim, dataset, seed=seed)
    decoder = get_decoder(input_dim, encoded_dim, dataset, seed=seed)
    inputs = Input(shape=(input_dim,))
    encoded = encoder(inputs)
    decoded = decoder(encoded)
    autoencoder = Model(inputs=inputs, outputs=decoded, name=f'{dataset}_autoencoder')
    return autoencoder

In [7]:
def train_autoencoder(x, autoencoder, epochs=100, batch_size=16, validation_split=0.1):
    """
    Train an autoencoder model with adam optimizer and mean squared error loss function.
    
    Parameters:
    x (array): Input data for training.
    autoencoder (Model): Autoencoder model to be trained.
    epochs (int): Number of training epochs.
    batch_size (int): Size of the training batches.
    validation_split (float): Fraction of the data to be used as validation data.
    
    Returns:
    History: Training history of the autoencoder.
    """
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    history = autoencoder.fit(x, x, epochs=100, batch_size=16, validation_split=0.1, verbose=0)
    return history

In [8]:
def compute_latent_encodings(x, autoencoder, dataset):
    """
    Compute the latent encodings using a trained autoencoder.
    
    Parameters:
    x (array): Input data to be encoded.
    autoencoder (Model): Trained autoencoder model.
    dataset (str): Name of the dataset.
    
    Returns:
    array: Latent encodings of the input data.
    """
    encoder = autoencoder.get_layer(f'{dataset}_encoder')
    trained_encodings = encoder(x).numpy()
    return trained_encodings

def visualise_latent_encodings(encodings1, y1, dataset1, encodings2, y2, dataset2, cmap1=viridis_cmap, cmap2=jet_cmap, 
                               txt='Figure 4: Latent 2D embeddings of the S-Curve (left) and Swiss Roll (right) '
                               'datasets from the trained autoencoders.'):
    """
    Generate scatter plots for the latent encodings of two datasets.
    
    Parameters:
    encodings1 (array): Latent encodings for the first dataset.
    y1 (array): Labels for the first dataset.
    dataset1 (str): Name of the first dataset.
    encodings2 (array): Latent encodings for the second dataset.
    y2 (array): Labels for the second dataset.
    dataset2 (str): Name of the second dataset.
    txt (str): Text for figure caption.
    cmap1 (Colormap): Colormap for the first dataset. Default to viridis_cmap.
    cmap2 (Colormap): Colormap for the second dataset. Default to jet_cmap.
    """
    fig = plt.figure(figsize=(8,3))
    
    # Plot the latent encodings for the first dataset
    plt.subplot(1,2,1)
    plt.scatter(encodings1[:, 0], encodings1[:, 1], c=y1, cmap=cmap1, alpha=0.3)
    plt.xlabel('Dim 1')
    plt.ylabel('Dim 2')
    plt.title(f'Latent Embeddings: {dataset1}')

    # Plot the latent encodings for the second dataset
    plt.subplot(1,2,2)
    plt.scatter(encodings2[:, 0], encodings2[:, 1], c=y2, cmap=cmap2, alpha=0.3)
    plt.xlabel('Dim 1')
    plt.ylabel('Dim 2')
    plt.title(f'Latent Embeddings: {dataset2}')
    fig.text(.5, .001, txt, ha='center')
    fig.tight_layout()
    plt.show()

## CNN Autoencoder

In [9]:
def get_cnn_encoder(input_shape, encoded_dim, dataset, seed=5):
    """
    Define the CNN encoder part of a CNN autoencoder.
    
    Parameters:
    input_shape (tuple): Shape of the input data.
    encoded_dim (int): Dimension of the encoded representation.
    dataset (str): Name of the dataset.
    seed (int): Random seed for weight initialization.
    
    Returns:
    Model: CNN encoder model.
    """
    inputs = Input(shape=(input_shape))
    x = Conv2D(32, 3, activation='relu', strides=2, padding='same', 
               kernel_initializer=GlorotUniform(seed=seed))(inputs)
    x = Conv2D(64, 3, activation='relu', strides=2, padding='same', 
               kernel_initializer=GlorotUniform(seed=seed))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu', kernel_initializer=GlorotUniform(seed=seed))(x)
    x = Dropout(0.2)(x)
    encoded = Dense(encoded_dim, kernel_initializer=GlorotUniform(seed=seed))(x)
    cnn_encoder = Model(inputs, encoded, name=f'{dataset}_cnn_encoder')
    return cnn_encoder

def get_cnn_decoder(input_shape, encoded_dim, dataset, seed=5):
    """
    Define the CNN decoder part of a CNN autoencoder.
    
    Parameters:
    input_shape (tuple): Shape of the output data.
    encoded_dim (int): Dimension of the encoded representation.
    dataset (str): Name of the dataset.
    seed (int): Random seed for weight initialization.
    
    Returns:
    Model: CNN decoder model.
    """
    encoded_inputs = Input(shape=(encoded_dim,))
    x = Dense(64, activation='relu', kernel_initializer=GlorotUniform(seed=seed))(encoded_inputs)
    x = Dense(7*7*64, activation='relu', kernel_initializer=GlorotUniform(seed=seed))(x)
    x = Reshape((7, 7, 64))(x)
    x = Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same', 
                        kernel_initializer=GlorotUniform(seed=seed))(x)
    decoded = Conv2DTranspose(input_shape[-1], 3, activation='sigmoid', strides=2, padding='same', 
                              kernel_initializer=GlorotUniform(seed=seed))(x)
    cnn_decoder = Model(encoded_inputs, decoded, name=f'{dataset}_cnn_decoder')
    return cnn_decoder
    
def get_cnn_autoencoder(input_shape, encoded_dim, dataset, seed=5):
    """
    Construct the complete CNN autoencoder by combining the CNN encoder and CNN decoder.
    
    Parameters:
    input_shape (tuple): Shape of the input data.
    encoded_dim (int): Dimension of the encoded representation.
    dataset (str): Name of the dataset.
    seed (int): Random seed for weight initialization.
    
    Returns:
    Model: CNN autoencoder model.
    """
    cnn_encoder = get_cnn_encoder(input_shape, encoded_dim, dataset, seed=seed)
    cnn_decoder = get_cnn_decoder(input_shape, encoded_dim, dataset, seed=seed)
    inputs = Input(shape=(input_shape))
    encoded = cnn_encoder(inputs)
    decoded = cnn_decoder(encoded)
    autoencoder = Model(inputs=inputs, outputs=decoded, name=f'{dataset}_cnn_autoencoder')
    return autoencoder

In [10]:
def train_cnn_autoencoder(x_train, x_test, cnn_autoencoder, epochs=100, batch_size=64, buffer_size =1000):
    """
    Train a CNN autoencoder model with adam optimizer and mean squared error loss function. 
    The model is trained with a early stopping if validation loss doesn't improve after 10 epochs.
    
    Parameters:
    x_train (array): Training data.
    x_test (array): Testing data.
    cnn_autoencoder (Model): CNN autoencoder model to be trained.
    epochs (int): Number of training epochs.
    batch_size (int): Size of the training batches.
    buffer_size (int): Buffer size for shuffling the training data.
    
    Returns:
    History: Training history of the CNN autoencoder.
    """
    # Create Tensorflow Dataset objects for train and test sets
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, x_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, x_test))
    
    # Shuffle and batch the datasets
    shuffle_buffer_size = buffer_size
    batch_size = batch_size
    train_dataset = train_dataset.shuffle(shuffle_buffer_size, seed).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    # Create an EarlyStopping callback to terminate training if the val_loss doesn't immprove after 10 epochs
    early_stopping = EarlyStopping(monitor='val_loss', 
                                   patience=10, 
                                   mode='min', 
                                   restore_best_weights=True)
    
    # Train the autoencoder with adam optimizer and mean squared error loss function with early stopping
    cnn_autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    history = cnn_autoencoder.fit(train_dataset, 
                                  epochs=epochs,
                                  validation_data=test_dataset, 
                                  callbacks=early_stopping, 
                                  verbose=0)

    return history

In [11]:
def reconstruct_ten_images(x, cnn_autoencoder, seed=19, 
                           txt='Figure 7: Comparison of 10 original (Top) vs. reconstructed (Bottom) '
                           'images randomly sampled from the MNIST test dataset.'):
    """
    Randomly sample and reconstruct 10 images using a trained CNN autoencoder. 
    Plot the original and reconstructed images for comparison.
    
    Parameters:
    x (array): Input images for reconstruction.
    cnn_autoencoder (Model): Trained CNN autoencoder model.
    seed (int): Random seed for reproducibility.
    txt (str): Text for figure caption.
    """
    # Set seed for reproducibility
    np.random.seed(seed)
    
    # Randomly sample and reconstruct 10 images
    inx = np.random.choice(x.shape[0], 10, replace=False)
    reconstructed_images = cnn_autoencoder(x[inx])

    # Examine reconstructions vs original images
    f, axs = plt.subplots(2, 10, figsize=(9, 2))
    for j in range(10):
        axs[0, j].imshow(x[inx][j], cmap='binary')
        axs[1, j].imshow(reconstructed_images[j].numpy().squeeze(), cmap='binary')
        axs[0, j].axis('off')
        axs[1, j].axis('off')
    f.text(.5, .001, txt, ha='center')
    plt.suptitle('MNIST Reconstructions', fontsize='medium')
    plt.show()

In [12]:
def examine_mnist_latent_encodings(x, y, cnn_autoencoder, n, 
                                   txt='Figure 8: Latent 2D embeddings of the MNIST dataset from '
                                   'the trained CNN autoencoder.'):
    """
    Compute and visualise the latent encodings for the MNIST dataset using a trained CNN autoencoder.
    
    Parameters:
    x (array): Input data to be encoded.
    y (array): Corresponding labels for the input data.
    cnn_autoencoder (Model): Trained CNN autoencoder model.
    n (int): Number of samples to visualize.
    txt (str): Text for figure caption.
    """
    # Commpute the trained latent encodings
    cnn_encoder = cnn_autoencoder.get_layer('mnist_cnn_encoder')
    trained_encodings = cnn_encoder(x).numpy()
    
    # Plot the latent encodings
    fig = plt.figure(figsize=(6,4))
    for i in set(y):
        plt.scatter(trained_encodings[:n][y[:n] == i, 0], 
                    trained_encodings[:n][y[:n] == i, 1], 
                    label=i, alpha=0.6)
    plt.xlabel('Dim 1')
    plt.ylabel('Dim 2')
    plt.title('Latent Embeddings: MNIST')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    fig.text(.5, .001, txt, ha='center')
    fig.tight_layout()
    plt.show()